In [1]:
import pandas as pd
import pyodbc
import useful_functions as use

In [5]:
username = 'vigrose'
password_edw = 'Hufflepuff~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)
password_aims = 'DQA8K@T7pXEqEt'
s = "DSN=aims_prod; UID={}; PWD={}".format(username, password_aims)
informix = pyodbc.connect(s)

In [2]:
ppd = use.get_ppd()

INFO:useful_functions:PPD file from Jan 16 2022
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (13,14,20,21,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
ppd.groupby('MD_DO_CODE').count()

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
MD_DO_CODE,,,,,,,,,,,,,,,,,,,,,
1,1208257,1208257,1208257,1186354,1208257,1208229,1208239,973512,37053,226124,...,917875,918756,906352,910148,280615,234728,280617,32654,25972,32655
2,100299,100299,100299,97064,100299,100296,100296,66670,1657,12686,...,75417,75412,74561,74849,15397,12885,15397,1185,944,1185


In [7]:
query = \
    '''
    SELECT 
    P.ENTITY_ID, 
    S.STUD_ID, 
    S.SCHOOL_ID, 
    S.EDU_STS, 
    S.GRAD_YR, 
    S.EDU_PREF_CD, 
    S.DEGREE_CD,
    S.END_DT
    FROM informix.PERSON_ET P, informix.SCHOOL_ATT_ST S 
    WHERE 
    P.CURR_PERSON_TYPE = 'S' 
    AND 
    P.ENTITY_ID = S.STUD_ID;
    '''
students = pd.read_sql(con=informix, sql=query)
students.head()

,entity_id,stud_id,school_id,edu_sts,grad_yr,edu_pref_cd,degree_cd,end_dt
0,6015941,6015941,14796570,WTH,1996,None,MD,1992-08-24
1,6016994,6016994,14796570,WTH,1996,None,MD,1992-09-30
2,6017111,6017111,14796570,WTH,1996,None,MD,1993-01-28
3,6018983,6018983,14796570,WTH,1998,None,MD,1997-02-10
4,6020621,6020621,14796570,DSM,1999,None,MD,1998-08-04


In [19]:
students[(students.grad_yr>2021)&(students.degree_cd=='DO   ')]

,entity_id,stud_id,school_id,edu_sts,grad_yr,edu_pref_cd,degree_cd,end_dt
36801,47415013,47415013,14797857,ENR,2022,NON,DO,None
38966,47531714,47531714,14798312,AFF,2022,NON,DO,None
42083,47196847,47196847,45877607,ENR,2022,NON,DO,None
45431,48047216,48047216,45877607,ENR,2022,NON,DO,None
46644,49813396,49813396,14797285,AFF,2022,NON,DO,None
...,...,...,...,...,...,...,...,...
166018,69840121,69840121,14797285,AFF,2022,NON,DO,None
166019,69841655,69841655,45713846,ENR,2023,NON,DO,None
166020,69842357,69842357,14796908,AFF,2025,NON,DO,None
166021,69843722,69843722,44394333,AFF,2022,NON,DO,None


In [15]:
students.degree_cd.unique()

array(['MD   ', 'DO   ', None], dtype=object)

In [20]:
informix.close()